<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 16


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----
Описание задачи:
Создать базовый класс PaymentMethod в C#, который будет представлять
различные способы оплаты. На основе этого класса разработать 2-3 производных
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из
классов должны быть реализованы новые атрибуты и методы, а также
переопределены некоторые методы базового класса для демонстрации
полиморфизма.

Требования к базовому классу PaymentMethod:

• Атрибуты: ID способа оплаты (PaymentMethodId), Название способа оплаты
(MethodName), Минимальная сумма (MinAmount).

• Методы:
o ProcessPayment(decimal amount): метод для обработки платежа
указанной суммы.
o CheckMinimumAmount(decimal amount): метод для проверки
минимальной суммы платежа.
o GetPaymentDetails(): метод для получения деталей способа оплаты.

Требования к производным классам:
1. ОнлайнОплата (OnlinePayment): Должен содержать дополнительные
атрибуты, такие как URL платежной системы (PaymentUrl).
Метод ProcessPayment() должен быть переопределен для включения URL
платежной системы в процесс оплаты.
2. БанковскийПеревод (BankTransfer): Должен содержать дополнительные
атрибуты, такие как Банковские данные (BankData).
Метод CheckMinimumAmount() должен быть переопределен для проверки
минимальной суммы платежа с учетом банковских комиссий.
3. Наличные (CashPayment) (если требуется третий класс): Должен содержать
дополнительные атрибуты, такие как Место выдачи наличных
(CashPickupPoint). Метод GetPaymentDetails() должен быть переопределен
для отображения места выдачи наличных.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

// Делегат для обработки события успешного платежа
public delegate void PaymentProcessedEventHandler(object sender, PaymentProcessedEventArgs e);

// Аргументы события успешного платежа
public class PaymentProcessedEventArgs : EventArgs
{
    public decimal Amount { get; set; }
    public string TransactionId { get; set; }
    public string Status { get; set; } // Добавлен атрибут Status
}


public class PaymentMethod
{
    private int _paymentMethodId;
    private string _methodName;
    private decimal _minAmount;
    private DateTime _paymentDate;
    private string _description;
    private bool _isCompleted;
    private string _currency;
    private List<PaymentHistory> _paymentHistory;
    private string _status; // Добавлен атрибут Status
    private decimal _fee; // Добавлен атрибут Fee

    public event PaymentProcessedEventHandler PaymentProcessed; // Событие успешного платежа


    public int PaymentMethodId 
    { 
        get {return _paymentMethodId;}
        set{ _paymentMethodId = value; } 
    }
    public string MethodName 
    { 
        get { return _methodName; } 
        set { _methodName = value; } 
    }

    public decimal MinAmount 
    { 
        get { return _minAmount; } 
        set { _minAmount = value; } 
    }

    public DateTime PaymentDate
    {
        get { return _paymentDate; }
        set { _paymentDate = value; }
    }

    public string Description
    {
        get { return _description; }
        set { _description = value; }
    }

    public bool IsCompleted
    {
        get { return _isCompleted; }
        set { _isCompleted = value; }
    }

    public string Currency 
    {
        get { return _currency; }
        set { _currency = value; }
    }

    public List<PaymentHistory> PaymentHistory // Свойство PaymentHistory
    {
        get { return _paymentHistory; }
        set { _paymentHistory = value; }
    }
    public string Status  // Свойство Status
    {
        get { return _status; }
        set { _status = value; }
    }
    public decimal Fee // Свойство Fee
    {
        get { return _fee; }
        set { _fee = value; }
    }


    public PaymentMethod(int paymentMethodId, string methodName, decimal minAmount, string currency)
    {
        PaymentMethodId = paymentMethodId;
        MethodName = methodName;
        MinAmount = minAmount;
        Currency = currency;
        PaymentHistory = new List<PaymentHistory>();
        Status = "Pending"; // Начальный статус - "Ожидание"
    }

    public virtual void ProcessPayment(decimal amount)
    {
        if (CheckMinimumAmount(amount))
        {
            Console.WriteLine($"Обработка платежа методом {MethodName} на сумму {amount} {Currency}");
            PaymentDate = DateTime.Now;
            IsCompleted = true;
            AddPaymentHistory(amount);
            Status = "Completed"; // Обновление статуса после успешной обработки
            OnPaymentProcessed(amount); // Вызов события
        }
    }

    public virtual bool CheckMinimumAmount(decimal amount)
    {
        if (amount < MinAmount)
        {
            Status = "Failed"; // Обновление статуса при неудачной обработке
            Console.WriteLine($"Сумма платежа меньше минимальной ({MinAmount} {Currency})");
            return false;
        }
        return true;
    }

    public virtual string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}, Статус: {Status}";
    }


    public virtual List<PaymentHistory> GetPaymentHistory()
    {
        return PaymentHistory;
    }

    private void AddPaymentHistory(decimal amount)
    {
        PaymentHistory.Add(new PaymentHistory
        {
            PaymentId = PaymentHistory.Count + 1,
            PaymentMethod = MethodName,
            Amount = amount,
            PaymentDate = PaymentDate
        });
    }

    protected virtual void OnPaymentProcessed(decimal amount)
    {
        PaymentProcessed?.Invoke(this, new PaymentProcessedEventArgs { Amount = amount, TransactionId = GetTransactionId(), Status = Status });
    }

    protected virtual string GetTransactionId()
    {
        return Guid.NewGuid().ToString();
    }

    public virtual void CancelPayment()
    {
        Console.WriteLine($"Платеж {MethodName} отменен.");
        Status = "Cancelled";
    }

    public virtual decimal CalculateFees(decimal amount)
    {
        return Fee;
    }
}



public class OnlinePayment : PaymentMethod
{
    private string _paymentUrl;
    private string _transactionId;
    private int _attemptsCount;
    private string _provider; // Добавлен атрибут Provider

    public string PaymentUrl 
    { 
        get  { return _paymentUrl; } 
        set { _paymentUrl = value; } 
    }
    public string TransactionId
    { 
        get  { return _transactionId; } 
        set { _transactionId = value; } 
    }
    public int AttemptsCount 
    { 
        get { return _attemptsCount; } 
        set { _attemptsCount = value; }
    }
    public string Provider 
    { 
       get { return _provider; } 
        set { _provider = value; }
    } // Свойство Provider

    public OnlinePayment(int paymentMethodId, string methodName, decimal minAmount, string paymentUrl, string currency, string provider) : base(paymentMethodId, methodName, minAmount, currency)
    {
        PaymentUrl = paymentUrl;
        AttemptsCount = 0;
        Provider = provider; // Инициализация Provider
    }

    public override void ProcessPayment(decimal amount)
    {
        base.ProcessPayment(amount);
        TransactionId = Guid.NewGuid().ToString();
        Console.WriteLine($"Перенаправление на платежную страницу: {PaymentUrl} (Provider: {Provider})");
        AttemptsCount++;
    }

    public string GetPaymentDetails(bool showTransactionId = false)
    {
        string details = base.GetPaymentDetails();
        if (showTransactionId)
        {
            details += $", ID транзакции: {TransactionId}";
        }
        return details;
    }

    public override decimal CalculateFees(decimal amount)
    {
        return amount * 0.02m; // 2% комиссия
    }
}

public class BankTransfer : PaymentMethod
{
    private string _bankData;
    private decimal _bankFee;
    private string _bankName;
    private bool _isBankDataVerified;
    private string _bankAccount; // Добавлен атрибут BankAccount

    public string BankData 
    { 
        get { return _bankData; } 
        set { _bankData = value; }
    }
    public decimal BankFee 
    { 
       get { return _bankFee; } 
        set { _bankFee = value; }
    }
    public string BankName 
    { 
        get { return _bankName; } 
        set { _bankName = value; } 
    }
    public bool IsBankDataVerified 
    { 
        get { return _isBankDataVerified; } 
        set { _isBankDataVerified = value; } 
    }
    public string BankAccount 
    { 
        get { return _bankAccount; } 
        set { _bankAccount = value; } 
    } // Свойство BankAccount

    public BankTransfer(int paymentMethodId, string methodName, decimal minAmount, string bankData, decimal bankFee, string bankName, string currency, string bankAccount) : base(paymentMethodId, methodName, minAmount, currency)
    {
        BankData = bankData;
        BankFee = bankFee;
        BankName = bankName;
        IsBankDataVerified = false;
        BankAccount = bankAccount; // Инициализация BankAccount
    }

    public override bool CheckMinimumAmount(decimal amount)
    {
        return base.CheckMinimumAmount(amount + BankFee);
    }

    public override string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}, Банковские данные: {BankData}, Банк: {BankName}, Комиссия: {BankFee}, Номер счета: {BankAccount}";
    }

    public string GetBankInfo()
    {
        return $"Название банка: {BankName}, Банковские данные: {BankData}";
    }

    public void VerifyBankData()
    {
        Console.WriteLine("Проверка банковских данных...");
        IsBankDataVerified = true;
    }

    public override decimal CalculateFees(decimal amount)
    {
        return BankFee;
    }
}


public class CashPayment : PaymentMethod
{
    private string _cashPickupPoint;
    private string _paymentReference;
    private bool _isPaymentRecorded;
    private string _employee; // Добавлен атрибут Employee

    public string CashPickupPoint 
    { 
        get { return _cashPickupPoint; } 
        set { _cashPickupPoint = value; } 
    }
    public string PaymentReference 
    { 
        get { return _paymentReference; } 
        set { _paymentReference = value; }
    }
    public bool IsPaymentRecorded 
    { 
        get { return _isPaymentRecorded; } 
        set { _isPaymentRecorded = value; }
    }
    public string Employee 
    { 
        get { return _employee; } 
        set { _employee = value; } 
    } // Свойство Employee

    public CashPayment(int paymentMethodId, string methodName, decimal minAmount, string cashPickupPoint, string currency, string employee) : base(paymentMethodId, methodName, minAmount, currency)
    {
        CashPickupPoint = cashPickupPoint;
        IsPaymentRecorded = false;
        Employee = employee; // Инициализация Employee
    }


    public string GetPaymentDetails(bool showReference = false)
    {
        string details = base.GetPaymentDetails();
        if (showReference && !string.IsNullOrEmpty(PaymentReference))
        {
            details += $", Номер платежа: {PaymentReference}";
        }
        return details + $", Сотрудник: {Employee}";
    }


    public void RecordPayment(string paymentReference)
    {
        PaymentReference = paymentReference;
        Console.WriteLine($"Платеж записан с номером: {paymentReference}");
        IsPaymentRecorded = true;
    }
}

public class PrepaidCard : OnlinePayment
{
    private string _cardNumber;
    private DateTime _cardExpiryDate;
    private bool _isCardVerified;
    private int _cvv; // Добавлен атрибут CVV

    public string CardNumber 
    { 
        get { return _cardNumber; } 
        set { _cardNumber = value; } 
    }
    public DateTime CardExpiryDate
    { 
        get { return _cardExpiryDate; } 
        set { _cardExpiryDate = value; }
    }
    public bool IsCardVerified 
    { 
       get { return _isCardVerified; } 
        set { _isCardVerified = value; } 
    }
    public int CVV 
    { 
        get { return _cvv; } 
        set { _cvv = value; } 
    } // Свойство CVV


    public PrepaidCard(int paymentMethodId, string methodName, decimal minAmount, string paymentUrl, string cardNumber, DateTime cardExpiryDate, string currency, string provider, int cvv) : base(paymentMethodId, methodName, minAmount, paymentUrl, currency, provider)
    {
        CardNumber = cardNumber;
        CardExpiryDate = cardExpiryDate;
        IsCardVerified = false;
        CVV = cvv; // Инициализация CVV
    }

    public override void ProcessPayment(decimal amount)
    {
        base.ProcessPayment(amount);
        Console.WriteLine($"Используется карта: {CardNumber}, Срок действия: {CardExpiryDate:dd/MM/yyyy}, CVV: {CVV}");
    }

    public bool IsCardExpired()
    {
        return CardExpiryDate < DateTime.Now;
    }

    public void VerifyCard()
    {
        Console.WriteLine("Проверка карты...");
        IsCardVerified = true;
    }

    public override decimal CalculateFees(decimal amount)
    {
        return base.CalculateFees(amount) + 1; // 2% + 1 фиксированная комиссия
    }
}



public interface ICommissionable
{
    decimal Commission { get; set; }
}

public class BankTransferWithCommission : BankTransfer, ICommissionable
{
    public decimal Commission { get; set; }

    public BankTransferWithCommission(int paymentMethodId, string methodName, decimal minAmount, string bankData, decimal bankFee, string bankName, string currency, string bankAccount, decimal commission) : base(paymentMethodId, methodName, minAmount, bankData, bankFee, bankName, currency, bankAccount)
    {
        Commission = commission;
    }
}

public class PaymentProcessor<T> where T : PaymentMethod
{
    public void ProcessPayment(T paymentMethod, decimal amount)
    {
        paymentMethod.ProcessPayment(amount);
    }
}


public class PaymentHistory
{
    public int PaymentId { get; set; }
    public string PaymentMethod { get; set; }
    public decimal Amount { get; set; }
    public DateTime PaymentDate { get; set; }
}


        OnlinePayment onlinePayment = new OnlinePayment(1, "Онлайн оплата", 100, "https://example.payment.com", "RUB", "Сбербанк");
        onlinePayment.PaymentProcessed += (sender, e) => Console.WriteLine($"Платеж обработан: сумма = {e.Amount}, ID транзакции = {e.TransactionId}, Статус = {e.Status}");
        PaymentProcessor<PaymentMethod> processor = new PaymentProcessor<PaymentMethod>();
        processor.ProcessPayment(onlinePayment, 200);
        Console.WriteLine(onlinePayment.GetPaymentDetails(true));
        Console.WriteLine($"Комиссия: {onlinePayment.CalculateFees(200)}");
        Console.WriteLine(onlinePayment.GetPaymentHistory().Count);


        BankTransfer bankTransfer = new BankTransfer(2, "Банковский перевод", 25, "Счет: 0987654321", 10, "ПриватБанк", "RUB", "1234567890");
        processor.ProcessPayment(bankTransfer, 150);
        Console.WriteLine(bankTransfer.GetPaymentDetails());
        Console.WriteLine($"Комиссия: {bankTransfer.CalculateFees(150)}");


        CashPayment cashPayment = new CashPayment(3, "Наличные", 50, "Офис компании", "RUB", "Иван Иванов");
        processor.ProcessPayment(cashPayment, 100);
        Console.WriteLine(cashPayment.GetPaymentDetails());
        cashPayment.RecordPayment("12345");
        Console.WriteLine(cashPayment.GetPaymentDetails(true));


        PrepaidCard prepaidCard = new PrepaidCard(4, "Оплата картой", 30, "https://example.payment.com", "00998877665544332211", DateTime.Now.AddYears(1), "RUB", "Мир", 123);
        processor.ProcessPayment(prepaidCard, 100);
        Console.WriteLine(prepaidCard.GetPaymentDetails(true));
        Console.WriteLine($"Комиссия: {prepaidCard.CalculateFees(100)}");
        prepaidCard.CancelPayment();


        BankTransferWithCommission bankTransferWithCommission = new BankTransferWithCommission(5, "Банковский перевод с комиссией", 25, "Счет: 9876543210", 10, "Сбербанк", "RUB", "0987654321", 25);
        processor.ProcessPayment(bankTransferWithCommission, 100);
        Console.WriteLine(bankTransferWithCommission.GetPaymentDetails());
        Console.WriteLine($"Комиссия: {bankTransferWithCommission.Commission}");

    

Обработка платежа методом Онлайн оплата на сумму 200 RUB
Платеж обработан: сумма = 200, ID транзакции = 93bb3e4f-3e09-4ab1-bae4-d3367bd92fd2, Статус = Completed
Перенаправление на платежную страницу: https://example.payment.com (Provider: Сбербанк)
Способ оплаты: Онлайн оплата, Минимальная сумма: 100 RUB, Статус: Completed, ID транзакции: 57d2efc7-0005-46b3-8dc1-21abef903a06
Комиссия: 4.00
1
Обработка платежа методом Банковский перевод на сумму 150 RUB
Способ оплаты: Банковский перевод, Минимальная сумма: 25 RUB, Банковские данные: Счет: 0987654321, Банк: ПриватБанк, Комиссия: 10, Номер счета: 1234567890
Комиссия: 10
Обработка платежа методом Наличные на сумму 100 RUB
Способ оплаты: Наличные, Минимальная сумма: 50 RUB, Статус: Completed, Сотрудник: Иван Иванов
Платеж записан с номером: 12345
Способ оплаты: Наличные, Минимальная сумма: 50 RUB, Статус: Completed, Номер платежа: 12345, Сотрудник: Иван Иванов
Обработка платежа методом Оплата картой на сумму 100 RUB
Перенаправление на плате